# Testing ML architectures implemented on the MLTSA package

In this packaged there are multiple architectures built in for testing on the different data available

In [1]:
"""First we import our dataset examples, and as usual generate data to work with"""
from OneD_pot_data import potentials
from OneD_pot_data import dataset

#This sets the potentials, don't re-run
total_n_pots = 25
n_DW = 5
relevant_DW_n = 2
#After defining the desired parameters we define the potentials accordingly
pots = potentials(total_n_pots, n_DW, relevant_DW_n)
# This creates the first dataset of data.
# It creates the mixing coefficients don't re-run
n_features = 180
degree_of_mixing = 2
#We specified the number of features wanted and how much they will mix
oneD_dataset = dataset(pots, n_features, degree_of_mixing)

Defining Potentials: 100%|##########| 25/25 [00:00<00:00, 217.97it/s]


In [2]:
"""Now we generate the trajectories we will use for the whole experiment"""
#Generate the trajectories
n_simulations = 100
n_steps = 250
data, ans = oneD_dataset.generate_linear(n_simulations, n_steps)
data_val, ans_val = oneD_dataset.generate_linear(int(n_simulations/2), n_steps)

#Prepare it for training
from sklearn.preprocessing import OneHotEncoder
time_frame = [30, 60] #Same time frame as the sklearn one
X, Y = oneD_dataset.PrepareData(data, ans, time_frame, mode="Normal")
X_val, Y_val = oneD_dataset.PrepareData(data_val, ans_val, time_frame, mode="Normal")

#We will convert IN and OUT to numerical labels
encoder = OneHotEncoder()
Y = encoder.fit_transform(Y.reshape(len(Y), 1)).toarray()
Y_val = encoder.fit_transform(Y_val.reshape(len(Y_val),1)).toarray()
print(Y)
print(Y_val)

Classifying Simulation Outcomes: 100%|##########| 50/50 [00:00<?, ?it/s]


Generating dataset
Getting simulation labels for the generated data
Generating dataset
Getting simulation labels for the generated data
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


Note that we will start by using the simple MLP, and we will move on to more advanced models.

First we will import the model builder, and we will try both the simple MLP and the more "deep" version with 5 layers.

In [3]:
#We will start with the basic Multi-Layer Perceptron
from MLTSA_tensorflow import TF_2_MLP

MLP = TF_2_MLP.build_MLP(n_steps, n_features, n_labels=2).model
MLP_deep = TF_2_MLP.build_MLP(n_steps, n_features, n_labels=2, type="deep").model

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Building Simple MLP
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               18100     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 28,402
Trainable params: 28,402
Non-trainable params: 0
______________________________________________

In [4]:
from tensorflow.keras.callbacks import EarlyStopping

MLP.fit(X, Y, epochs=500, batch_size=n_steps, verbose=1, validation_split=0.2,
        callbacks=[EarlyStopping(monitor='loss', min_delta=1e-8, restore_best_weights=True, patience=50)])
val_acc = MLP.evaluate(X_val, Y_val, verbose=1)
print("We achieved", val_acc[1]*100, "% accuracy on Validation")

Train on 2400 samples, validate on 600 samples
Epoch 1/500
2400/2400 [==============================] - 0s 91us/sample - loss: 0.6989 - acc: 0.5117 - val_loss: 0.6980 - val_acc: 0.5000
Epoch 2/500
2400/2400 [==============================] - 0s 16us/sample - loss: 0.7008 - acc: 0.5004 - val_loss: 0.6937 - val_acc: 0.5000
Epoch 3/500
2400/2400 [==============================] - 0s 16us/sample - loss: 0.6945 - acc: 0.5183 - val_loss: 0.6935 - val_acc: 0.5000
Epoch 4/500
2400/2400 [==============================] - 0s 16us/sample - loss: 0.6958 - acc: 0.4992 - val_loss: 0.6945 - val_acc: 0.5000
Epoch 5/500
2400/2400 [==============================] - 0s 16us/sample - loss: 0.6924 - acc: 0.5213 - val_loss: 0.6926 - val_acc: 0.5633
Epoch 6/500
2400/2400 [==============================] - 0s 17us/sample - loss

In [6]:
#A deep learning model needs more epochs to train to get to convergence. We will also allow more patience
MLP_deep.fit(X, Y, epochs=1000, batch_size=n_steps, verbose=1, validation_split=0.2,
        callbacks=[EarlyStopping(monitor='loss', min_delta=1e-8, restore_best_weights=True, patience=100)])
val_acc = MLP_deep.evaluate(X_val, Y_val, verbose=1)
print("We achieved", val_acc[1]*100, "% accuracy on Validation")

Train on 2400 samples, validate on 600 samples
Epoch 1/1000
2400/2400 [==============================] - 0s 35us/sample - loss: 0.7035 - acc: 0.5392 - val_loss: 0.6990 - val_acc: 0.5000
Epoch 2/1000
2400/2400 [==============================] - 0s 36us/sample - loss: 0.6949 - acc: 0.5125 - val_loss: 0.6952 - val_acc: 0.5000
Epoch 3/1000
2400/2400 [==============================] - 0s 34us/sample - loss: 0.6932 - acc: 0.5125 - val_loss: 0.6939 - val_acc: 0.5000
Epoch 4/1000
2400/2400 [==============================] - 0s 35us/sample - loss: 0.6928 - acc: 0.5125 - val_loss: 0.6935 - val_acc: 0.5000

In [7]:
#We will start with the basic LSTM ("Vanilla")
from MLTSA_tensorflow import TF_2_LSTM

LSTM = TF_2_LSTM.build_LSTM(n_steps, n_features, n_labels=2).model


Building LSTM
Multilabel classification model
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               112400    
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 202       
Total params: 112,602
Trainable params: 112,602
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
LSTM.fit(X, Y, epochs=1000, batch_size=n_steps, verbose=1, validation_split=0.2,
        callbacks=[EarlyStopping(monitor='loss', min_delta=1e-8, restore_best_weights=True, patience=100)])
val_acc = MLP_deep.evaluate(X_val, Y_val, verbose=1)
print("We achieved", val_acc[1]*100, "% accuracy on Validation")

ValueError: Error when checking input: expected lstm_input to have 3 dimensions, but got array with shape (3000, 180)